# FastF1 Telemetry

### This file contains scripts that serve as ETL footprints for the VisionF1 telemetry data pipeline. They obtain data from the API or the local cache, process it as needed, and the objective is to then upload it to the VisionF1 database (MongoDB Atlas).

## Race Pace

All data from 2018 to now.

In [ ]:
import fastf1
import pandas as pd
from datetime import datetime

SEASONS = range(2018, datetime.now().year + 1)

all_race_pace = []

for season in SEASONS:
    schedule = fastf1.get_event_schedule(season)
    schedule = schedule[schedule["EventFormat"] != "testing"]

    for _, event in schedule.iterrows():
        event_name = event["EventName"]
        round_number = event["RoundNumber"]
        try:
            session = fastf1.get_session(season, event_name, 'R')
            session.load()
        except Exception as e:
            print(f"Error loading session {season} {event_name}.")
            continue

        if not hasattr(session, '_laps'):
            print(f"No lap data for {season} {event_name}, skipping.")
            continue
        
        laps = session.laps
        laps = laps[laps['IsAccurate']]
        drivers = laps['Driver'].unique()

        for driver in drivers:
            filtered_laps = laps.pick_drivers([driver])
            
            avg_laptime = filtered_laps["LapTime"].mean()
            std_laptime = filtered_laps["LapTime"].std()

            all_race_pace.append({
                "season": season,
                "round": int(round_number),
                "event": event_name,
                "driver": driver,
                "avg_laptime": avg_laptime.total_seconds(),
                "std_laptime": std_laptime.total_seconds() if pd.notnull(std_laptime) else None,
                "race_pace_id": f"{season}_{round_number}_{driver}"
            })

df_race_pace = pd.DataFrame(all_race_pace)

In [48]:
df_race_pace

,season,round,event,driver,avg_laptime,std_laptime,race_pace_id
0,2018,1,Australian Grand Prix,GAS,91.997111,2.371715,2018_1_GAS
1,2018,1,Australian Grand Prix,PER,89.530956,1.307663,2018_1_PER
2,2018,1,Australian Grand Prix,ALO,89.019833,1.336029,2018_1_ALO
3,2018,1,Australian Grand Prix,LEC,90.349555,1.503345,2018_1_LEC
4,2018,1,Australian Grand Prix,STR,90.460340,1.141896,2018_1_STR
...,...,...,...,...,...,...,...
3191,2025,18,Singapore Grand Prix,COL,98.863120,1.176053,2025_18_COL
3192,2025,18,Singapore Grand Prix,BOR,98.736327,1.050940,2025_18_BOR
3193,2025,18,Singapore Grand Prix,OCO,98.871396,1.067156,2025_18_OCO
3194,2025,18,Singapore Grand Prix,GAS,98.458625,1.555571,2025_18_GAS


In [58]:
ver = df_race_pace[ df_race_pace["driver"] == "VER" ]
ver_2025 = ver[ ver["season"] == 2025 ]
ver_2025

,season,round,event,driver,avg_laptime,std_laptime,race_pace_id
2851,2025,1,Australian Grand Prix,VER,90.314777,2.618743,2025_1_VER
2871,2025,2,Chinese Grand Prix,VER,97.199452,1.050277,2025_2_VER
2888,2025,3,Japanese Grand Prix,VER,92.459200,0.927919,2025_3_VER
2913,2025,4,Bahrain Grand Prix,VER,98.083375,0.960517,2025_4_VER
2929,2025,5,Saudi Arabian Grand Prix,VER,93.191066,0.582681,2025_5_VER
2946,2025,6,Miami Grand Prix,VER,91.751291,1.136319,2025_6_VER
2965,2025,7,Emilia Romagna Grand Prix,VER,80.305176,1.168307,2025_7_VER
2985,2025,8,Monaco Grand Prix,VER,75.852114,1.453537,2025_8_VER
3005,2025,9,Spanish Grand Prix,VER,79.654830,1.071211,2025_9_VER
3024,2025,10,Canadian Grand Prix,VER,75.547278,0.688125,2025_10_VER


In [69]:
race_pace_2025 = df_race_pace[ df_race_pace["season"] == 2025 ]
race_pace_2025_15 = race_pace_2025[ race_pace_2025["round"] == 15 ]
race_pace_2025_15 = race_pace_2025_15.sort_values('avg_laptime')
dict = race_pace_2025_15.to_dict(orient="records")
print(dict)

[{'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'PIA', 'avg_laptime': 74.48166, 'std_laptime': 0.98182, 'race_pace_id': '2025_15_PIA'}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'NOR', 'avg_laptime': 74.665288, 'std_laptime': 0.920907, 'race_pace_id': '2025_15_NOR'}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'VER', 'avg_laptime': 75.104875, 'std_laptime': 1.047516, 'race_pace_id': '2025_15_VER'}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'HAD', 'avg_laptime': 75.201928, 'std_laptime': 0.968081, 'race_pace_id': '2025_15_HAD'}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'RUS', 'avg_laptime': 75.55991, 'std_laptime': 0.893123, 'race_pace_id': '2025_15_RUS'}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'ANT', 'avg_laptime': 75.635259, 'std_laptime': 1.167985, 'race_pace_id': '2025_15_ANT'}, {'season': 2025, 'round': 15, 'event': 'Dutch Gr

In [75]:
session = fastf1.get_session(2025, 'Dutch Grand Prix', 'R')
session.load()
fastf1.plotting.get_driver_color_mapping(session=session)

core           INFO 	Loading data for Dutch Grand Prix - Race [v3.5.3]
2025-10-19 15:49:18,937 INFO Loading data for Dutch Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
2025-10-19 15:49:18,939 INFO Using cached data for session_info
req            INFO 	Using cached data for driver_info
2025-10-19 15:49:18,940 INFO Using cached data for driver_info
2025-10-19 15:49:18,947 DEBUG Failed to parse timestamp '-1:47:38.638' in Ergastresponse.
req            INFO 	Using cached data for session_status_data
2025-10-19 15:49:18,966 INFO Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
2025-10-19 15:49:18,968 INFO Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
2025-10-19 15:49:18,969 INFO Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
2025-10-19 15:49:18,998 INFO Using cached data for _extended_timing_data
req   

{'VER': '#0600ef',
 'TSU': '#0600ef',
 'GAS': '#ff87bc',
 'COL': '#ff87bc',
 'ANT': '#27f4d2',
 'RUS': '#27f4d2',
 'ALO': '#00665f',
 'STR': '#00665f',
 'LEC': '#e80020',
 'HAM': '#e80020',
 'ALB': '#00a0dd',
 'SAI': '#00a0dd',
 'HUL': '#00e700',
 'BOR': '#00e700',
 'LAW': '#fcd700',
 'HAD': '#fcd700',
 'OCO': '#b6babd',
 'BEA': '#b6babd',
 'NOR': '#ff8000',
 'PIA': '#ff8000'}